In [ ]:
# run extract.bat "Path\to\SignalExtraction\Tool" "Path\to\method_data\root_directory" to automate SignalExtraction. 

# this takes a while and provides output only once complete or interrupted, unless the subprocess library is used to capture the shell output in real time
#!src\StreamPort\extract.bat "C:\Users\Sandeep\Desktop\SignalExtraction v.01" "C:\Users\Sandeep\Desktop\Error-LC\Method-Data"

In [ ]:
import os
from src.StreamPort.device.analyses import PressureCurvesAnalyses

path = "C:/Users/Sandeep/Desktop/Error-LC/Method-Data"
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]

error_lc_files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    error_lc_files.extend(batch_files)

#%store error_lc_files    
analyses = PressureCurvesAnalyses(files=error_lc_files)
print("Number of analyses: ", len(analyses.data))

In [ ]:
# error_lc curves belong to the same method
batches = analyses.get_batches()
batch_plot = analyses.plot_batches()
batch_plot.show()
print(batches)

In [ ]:
import pandas as pd

batch_list = pd.read_excel("C:/Users/Sandeep/Desktop/Error-LC/Batch-Liste.xlsx")
batch_list.head()

bad_batch_ids = batch_list["provoked_error"] != None
bad_batches = batch_list.loc[bad_batch_ids, "batch_name"]
print(bad_batches)

In [ ]:
batch_indices = []
for i in batches:
    indices = analyses.get_batch_indices(i)
    print("batch: ", i," indices: ", indices)
    batch_indices.extend(indices)

In [ ]:
print("Number of curves: ", len(batch_indices))

In [ ]:
train_batch = analyses.get_batch_indices('250613_Mix-1_10x10ng-mL 2025-06-13 14-40-12') + (analyses.get_batch_indices('250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43')) + (analyses.get_batch_indices('250616_Mix-1_50x100ng-mL 2025-06-16 10-06-17')) + (analyses.get_batch_indices('250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58')) + (analyses.get_batch_indices('250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42')) + (analyses.get_batch_indices('250626_Mix-1_50x100ng-mL 2025-06-26 09-05-34')) + (analyses.get_batch_indices('250702_Mix-1+IS_50x100ng-mL 2025-07-02 09-46-06')) + (analyses.get_batch_indices('250707_Mix-1+IS_30x100ng-mL 2025-07-07 14-47-33'))

In [ ]:
fig_train_curves_raw = analyses.plot_pressure_curves(analyses.get_batch_indices("250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42")+ (analyses.get_batch_indices("250624_Mix-1_30x100ng-mL 2025-06-24 10-33-02")) + analyses.get_batch_indices("250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58"))#train_batch[ : len(train_batch)//2])

for trace in fig_train_curves_raw.data:
    trace.line.color = "black"

fig_train_curves_raw.show()

In [ ]:
for i in [612, 613, 616, 366, 367, 369, 368, 31, 615, 614, 16, 0, 1, 13, 15]:
    if i in train_batch:
        train_batch.remove(i)

In [ ]:
for i in range(189, 217):
    if i in train_batch:
        train_batch.remove(i)
for j in range(222, 288):
    if j in train_batch:
        train_batch.remove(j)

In [ ]:
test_batch = list(set(batch_indices) - set(train_batch))
print("Train: ", len(train_batch), " ", train_batch)
print("Test: ", len(test_batch), " ", (test_batch))

In [ ]:
fig_train_curves = analyses.plot_pressure_curves(train_batch)
fig_train_curves.show()

In [ ]:
import random
import datetime

train_set = random.sample(train_batch, random.randint(20, 35))
print("Train set: ", len(train_set), " curves - ", train_set)
test_dates = []

for i in [1, 2, 3]:
    now = datetime.datetime.now().isoformat()
    now = now.replace(":", "-").replace(".", "-").replace("T", " ")

    test_set = random.sample(test_batch, len(train_set)//3)#take ~25-35% of train data size for test data
    test_dates.append({now : test_set})
    print("Test set ", now ,  " indices: ", test_set)


In [ ]:
fig_train_curves_raw = analyses.plot_pressure_curves(train_set)

for trace in fig_train_curves_raw.data:
    trace.line.color = "black"

fig_train_curves_raw.show()

In [ ]:
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative

processor = PressureCurvesMethodExtractFeaturesNative(window_size=random.randint(3, 11), bins=random.randint(3, 8)) # defaults: (period = 10, window_size = 7, bins = 4, crop = 2)
processor.run(analyses)
parameters = processor.parameters
print("Feature extraction parameters: ", parameters)

In [ ]:
fig_features=analyses.plot_features(indices = train_set)

for trace in fig_features.data:
    trace.line.color = "black"

fig_features.show()

In [ ]:
train_data = analyses.get_features(train_set)
train_metadata = analyses.get_metadata(train_set)
train_data.to_csv("dev/error_lc_train_features.csv", index=False)
train_metadata.to_csv("dev/error_lc_train_metadata.csv", index=False)
print("Number of training curves: ", len(train_set))

In [ ]:
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

iso_analyses = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(iso_analyses)

In [ ]:
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn 

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type=random.choice(["RobustScaler", "MaxAbsScaler", "MaxNormalizer", "StandardScaler", "MinMaxScaler"]))
scaling_parameters = scl.parameters
print("Scaling parameters: ", scaling_parameters)
parameters.update(scaling_parameters)

iso_analyses = scl.run(iso_analyses)
fig_train_features = iso_analyses.plot_data()
fig_train_features.update_layout(title="Train set features")
fig_train_features.show()

In [ ]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iso = MachineLearningMethodIsolationForestSklearn()#force determinism to explain feature importance or for clean model generalization
iforest_parameters = iso.parameters
print("Isolation Forest parameters: ", iforest_parameters)
parameters.update(iforest_parameters)

iso_analyses = iso.run(iso_analyses)
iso_analyses.train()
fig_train_scores = iso_analyses.plot_scores()
fig_train_scores.show()

parameters_df = []

for test in test_dates:
    date = list(test.keys())[0]
    test_set = list(test.values())[0]

    test_data = analyses.get_features(test_set)
    test_metadata = analyses.get_metadata(test_set)
    test_data.to_csv(f"dev/error_lc_test_{date}_features.csv", index=False)
    test_metadata.to_csv(f"dev/error_lc_test_{date}_metadata.csv", index=False)

    fig_test_curves=analyses.plot_pressure_curves(indices = test_set)
    fig_test_curves.update_layout(showlegend=False)
    fig_test_curves.write_image(f"dev/figures/fig_error_lc_test_{date}_curves.png", width=1000, height= 350, scale = 3)
    #fig_test_curves.show()

    fig_test_features=analyses.plot_features(indices = test_set)
    fig_test_features.update_layout(showlegend=False)
    fig_test_features.write_image(f"dev/figures/fig_error_lc_test_{date}_features.png", width=1100, height= 350, scale = 3)
    fig_test_features.show()
    
    fig_test_features_raw = analyses.plot_features_raw(indices = test_set)
    fig_test_features_raw.update_layout(showlegend=False)
    fig_test_features_raw.write_image(f"dev/figures/fig_error_lc_test_{date}_features_raw.png", width=1100, height= 350, scale = 3)
    fig_test_features_raw.show()



    iso_analyses.predict(test_data, test_metadata)
    outliers_test = iso_analyses.test_prediction_outliers()

    classified_test_metadata = outliers_test
    classified_test_metadata["date"] = date

    print("Test set ", i, ": \n", test_metadata["index"].tolist()) 
    print("\n")
    
    test_batch_parameters = parameters.copy()
    test_size = len(test_metadata)

    train_scores = iso_analyses.get_training_scores()
    train_size = len(train_scores)

    threshold = classified_test_metadata["threshold"].iloc[0]
    num_outliers = sum(classified_test_metadata["class"] == "outlier")
    percent_outliers = (sum(classified_test_metadata["class"] == "outlier")/len(classified_test_metadata["class"]))*100

    fig_test_scores = iso_analyses.plot_scores()
    fig_test_scores.write_image(f"dev/figures/fig_error_lc_test_{date}_scores.png", width=1100, height= 350, scale = 3)
    fig_test_scores.update_layout(title=f"Test set {date[:19].replace("T", " ")}")
    fig_test_scores.show()

    fig_test_features = iso_analyses.plot_data()
    fig_test_features.write_image(f"dev/figures/fig_error_lc_test_{date}_features.png", width=1100, height= 350, scale = 3)
    #fig_test_features.show()

    #optionally add seen normal curves to train set
    #iso_analyses.add_prediction()

    # if num_outliers < 1:
    #     threshold += 0.5*np.std(train_scores)

    test_batch_parameters.update(
        {
        "date" : date,
        "train_set" : train_size, 
        "test_set" : test_size,
        "threshold" : threshold,
        "outliers" : num_outliers,
        "outliers_percent" : percent_outliers
        }
    )
    parameters_df.append(test_batch_parameters)
    classified_test_metadata.to_csv(f"dev/error_lc_test_{date}_classified_samples.csv", index=False)

In [ ]:

parameters_df = pd.DataFrame(parameters_df)
if os.path.exists("dev/error_lc_test_record.csv"):
    old_records = pd.read_csv("dev/error_lc_test_record.csv")
    parameters_df = pd.concat([old_records, parameters_df])
    parameters_df.drop_duplicates(subset=["date"], keep="last", inplace=True)
    
parameters_df.to_csv("dev/error_lc_test_record.csv", index = False)
print("Workflow parameters: \n", parameters_df)

In [ ]:
import plotly.graph_objects as go

test_record = pd.read_csv("dev/error_lc_test_record.csv") if os.path.exists("dev/error_lc_test_record.csv") else None
test_record = test_record.sort_values("date")

if test_record is not None:
    result_logs = []
    for date in test_record["date"]:
        result_logs.append(f"dev/error_lc_test_{date}_classified_samples.csv") if os.path.exists(f"dev/error_lc_test_{date}_classified_samples.csv") else print(f"No records for {date}")
    
    result_logs = [pd.read_csv(log) for log in result_logs]

    for log in result_logs:
        log["date"] = log["date"].astype(str).str[:19].str.replace("T", " ")

    result_logs = [log.to_string(index=False).replace("\n", "<br>") for log in result_logs]

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=test_record["date"],
            y=test_record["threshold"],
            mode="lines+markers",
            name="Threshold",
            yaxis="y1",  
            hovertemplate=[
                "<br>Threshold: " + str(test_record["threshold"][i])  
                for i in range(len(test_record))
            ],
            line=dict(color="red", width=2, dash='dash'),
            marker=dict(size=8, symbol="circle")
        )
    )

    fig.add_trace(
        go.Bar(
            x=test_record["date"],
            y=test_record["outliers"],
            name="Outliers",
            yaxis="y2",
            width = 0.15, #width
            marker_color="blue",
            hovertext=result_logs,
            hoverinfo="text"
        )
    )

    # fig.add_trace(
    #     go.Scatter(
    #         x=test_record["date"],
    #         y=test_record["outliers"],
    #         mode="lines+markers",
    #         name="Outliers",
    #         yaxis="y2",
    #         line=dict(color="blue", width=2),
    #         marker=dict(size=8, symbol="diamond", color="blue"),
    #         hovertemplate=[
    #             "<br>Train Set: " + str(test_record["train_set"][i]) +
    #             "<br>Test Set: " + str(test_record["test_set"][i]) +
    #             "<br>Outliers: " + str(test_record["outliers"][i])
    #             for i in range(len(test_record))
    #         ]
    #     )
    # )

    fig.add_trace(
        go.Scatter(
            x=test_record["date"],
            y=test_record["train_set"], 
            mode="lines+markers",
            name="Training curves",
            yaxis="y2",
            hovertemplate=[
                "<br>Training samples: " + str(test_record["train_set"][i]) +
                "<br>Test samples: " + str(test_record["test_set"][i]) +
                "<br>Outliers: " + str(test_record["outliers"][i]) +
                "<br>Outliers %: " + str(test_record["outliers_percent"][i]) 
                for i in range(len(test_record))
            ],
            line=dict(color="green", width=2, dash='solid'),
            marker=dict(size=8, symbol="star")
        )
    )

fig.update_layout(
    title="Detection accuracy over Test Runs and Training Set Size",
    xaxis=dict(
        tickvals=test_record["date"], 
        ticktext=[d[:19].replace("T", " ") for d in test_record["date"]], 
        tickangle=45,
        title="Test Dates"
    ),
    yaxis=dict(  
        title=dict(text="Threshold", font=dict(color="red")),
        tickfont=dict(color="red")
    ),
    yaxis2=dict(#train set and outlier realistic in scale. Uncomment y3 to adjust 
        title=dict(text="Outliers", font=dict(color="blue")),
        tickfont=dict(color="blue"),
        overlaying="y",
        side="right"
    ),
    bargap = 1, 
    template="simple_white",
    legend=dict(
        x=0.5,
        y=1.1,
        xanchor="center",
        yanchor="top",
        orientation="h",  
        bgcolor="rgba(255,255,255,0.5)", 
        borderwidth=1  
    )
)
fig.write_image("dev/figures/fig_error_lc_threshold_variation.png", width=1100, height= 350, scale= 3)
fig.show()
